In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
from collections import Counter
df = pd.read_csv('../datasets/movie_data.csv', encoding='utf-8')
df

2024-08-09 02:35:37.446143: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 02:35:37.446599: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 02:35:37.448412: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 02:35:37.454044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 02:35:37.463804: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
...,...,...
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0


In [2]:
# 데이터셋 객체 생성
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices((df.values, target.values))

# 데이터셋 확인
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


I0000 00:00:1723138538.843931  283543 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-09 02:35:38.844178: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-08-09 02:35:38.902422: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
df.shape

(50000, 1)

In [3]:
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(target.shape[0], reshuffle_each_iteration=False)

ds_raw = ds_raw.take(100)
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'Alright, let me break it down for ya... Haggard is' 1
b'I was impressed by the beautiful photography in th' 1
b"I've just watched Fingersmith, and I'm stunned to " 0


2024-08-09 02:35:38.960146: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
import myprocess  # 'myprocess.py' 파일을 import합니다.

preprocessor = myprocess.PreProcess(ds_raw, 100)

In [5]:
batch_size = 10

train_data, valid_data, test_data = preprocessor.get_datas(batch_size, train_size=0.4, valid_size = 0.1, test_size=0.5)

In [6]:
# for ex in ds_train.shuffle(10).take(5):
#     print('sequence length', ex[0].shape)

# print('어휘 사전 크기', len(preprocessor.token_counts))

In [7]:
vocab_size = len(preprocessor.token_counts)+ 2  # 예시를 위해 설정한 값, 실제 vocab_size에 맞게 변경 필요
node_nums = [5, 10, 15]
embedding_dims = [20, 40]
layer_nums = [1, 2]

model_list= myprocess.make_models(vocab_size, node_nums, embedding_dims, layer_nums)

In [8]:
for i, model in enumerate(model_list):
    print(f"Model {i+1} Summary:")
    # model.summary()

Model 1 Summary:
Model 2 Summary:
Model 3 Summary:
Model 4 Summary:
Model 5 Summary:
Model 6 Summary:
Model 7 Summary:
Model 8 Summary:
Model 9 Summary:
Model 10 Summary:
Model 11 Summary:
Model 12 Summary:
Model 13 Summary:
Model 14 Summary:
Model 15 Summary:
Model 16 Summary:
Model 17 Summary:
Model 18 Summary:
Model 19 Summary:
Model 20 Summary:
Model 21 Summary:
Model 22 Summary:
Model 23 Summary:
Model 24 Summary:
Model 25 Summary:
Model 26 Summary:
Model 27 Summary:


In [9]:
myprocess.compile_train_save(model_list[:4], 5, train_data, valid_data, test_data  )

Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 126ms/step - accuracy: 0.5950 - loss: 0.6922 - val_accuracy: 0.6000 - val_loss: 0.6814
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.5950 - loss: 0.6828 - val_accuracy: 0.6000 - val_loss: 0.6748
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5950 - loss: 0.6758 - val_accuracy: 0.6000 - val_loss: 0.6676
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5950 - loss: 0.6681 - val_accuracy: 0.6000 - val_loss: 0.6589
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.5950 - loss: 0.6587 - val_accuracy: 0.6000 - val_loss: 0.6482


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6100 - loss: 0.6487
Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 126ms/step - accuracy: 0.5536 - loss: 0.6928 - val_accuracy: 0.6000 - val_loss: 0.6902
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5950 - loss: 0.6907 - val_accuracy: 0.6000 - val_loss: 0.6866
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5950 - loss: 0.6875 - val_accuracy: 0.6000 - val_loss: 0.6826
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.5950 - loss: 0.6831 - val_accuracy: 0.6000 - val_loss: 0.6772
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5950 - loss: 0.6769 - val_accuracy: 0.6000 - val_loss: 0.6655


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6100 - loss: 0.6662
Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.5950 - loss: 0.6930 - val_accuracy: 0.6000 - val_loss: 0.6916
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5950 - loss: 0.6917 - val_accuracy: 0.6000 - val_loss: 0.6904
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5950 - loss: 0.6905 - val_accuracy: 0.6000 - val_loss: 0.6891
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.5950 - loss: 0.6892 - val_accuracy: 0.6000 - val_loss: 0.6875
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5950 - loss: 0.6877 - val_accuracy: 0.6000 - val_loss: 0.6856


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6100 - loss: 0.6855
Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.4631 - loss: 0.6946 - val_accuracy: 0.6000 - val_loss: 0.6925
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.5950 - loss: 0.6906 - val_accuracy: 0.6000 - val_loss: 0.6891
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.5950 - loss: 0.6869 - val_accuracy: 0.6000 - val_loss: 0.6860
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5950 - loss: 0.6830 - val_accuracy: 0.6000 - val_loss: 0.6826
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5950 - loss: 0.6789 - val_accuracy: 0.6000 - val_loss: 0.6779


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6100 - loss: 0.6740


([{'accuracy': [0.5600000023841858,
    0.5600000023841858,
    0.5600000023841858,
    0.5600000023841858,
    0.5600000023841858],
   'loss': [0.6943395733833313,
    0.6866958737373352,
    0.6797022223472595,
    0.6714575886726379,
    0.6614041328430176],
   'val_accuracy': [0.6000000238418579,
    0.6000000238418579,
    0.6000000238418579,
    0.6000000238418579,
    0.6000000238418579],
   'val_loss': [0.6813910603523254,
    0.6747540235519409,
    0.6675747036933899,
    0.6588791012763977,
    0.648232102394104]},
  {'accuracy': [0.5400000214576721,
    0.5600000023841858,
    0.5600000023841858,
    0.5600000023841858,
    0.5600000023841858],
   'loss': [0.692973792552948,
    0.6912850737571716,
    0.6883357167243958,
    0.6839531064033508,
    0.678098738193512],
   'val_accuracy': [0.6000000238418579,
    0.6000000238418579,
    0.6000000238418579,
    0.6000000238418579,
    0.6000000238418579],
   'val_loss': [0.6902070045471191,
    0.6865905523300171,
    0.68260

In [12]:
model_list

[<Sequential name=sequential, built=True>,
 <Sequential name=sequential_1, built=True>,
 <Sequential name=sequential_2, built=True>,
 <Sequential name=sequential_3, built=True>,
 <Sequential name=sequential_4, built=False>,
 <Sequential name=sequential_5, built=False>,
 <Sequential name=sequential_6, built=False>,
 <Sequential name=sequential_7, built=False>,
 <Sequential name=sequential_8, built=False>,
 <Sequential name=sequential_9, built=False>,
 <Sequential name=sequential_10, built=False>,
 <Sequential name=sequential_11, built=False>,
 <Sequential name=sequential_12, built=False>,
 <Sequential name=sequential_13, built=False>,
 <Sequential name=sequential_14, built=False>,
 <Sequential name=sequential_15, built=False>,
 <Sequential name=sequential_16, built=False>,
 <Sequential name=sequential_17, built=False>,
 <Sequential name=sequential_18, built=False>,
 <Sequential name=sequential_19, built=False>,
 <Sequential name=sequential_20, built=False>,
 <Sequential name=sequential_

In [13]:
for model in model_list[:4]:
    test_results = model.evaluate(test_data)
    print('test accuracy : {:.2f}%'. format(test_results[1]*100))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6100 - loss: 0.6487
test accuracy : 60.00%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6100 - loss: 0.6662
test accuracy : 60.00%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6100 - loss: 0.6855
test accuracy : 60.00%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6100 - loss: 0.6740
test accuracy : 60.00%


In [14]:
model_list.clear()

In [16]:
model_list = myprocess.load_models_from_files(num_models=4)

In [17]:
for model in model_list:
    test_results = model.evaluate(test_data)
    print('test accuracy : {:.2f}%'. format(test_results[1]*100))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6100 - loss: 0.6487 
test accuracy : 60.00%
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6100 - loss: 0.6662
test accuracy : 60.00%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6100 - loss: 0.6855 
test accuracy : 60.00%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6100 - loss: 0.6740 
test accuracy : 60.00%


In [19]:
y_pred = myprocess.majority_vote_ensemble(model_list, test_data, accuracy=0.5)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
